In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pylab as pl
import os
import plotly
from fiona.crs import from_epsg
from geopandas.tools import sjoin
from IPython.display import display, clear_output
import time

% pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# REad in Schmelt form data
schmelt = pd.read_csv('data/Schmelt_bg.csv', index_col=0)

# Ensure formated as numerical
schmelt[['1996', '1997', '1998', '1999', '2000', '2001',
         '2002', '2003', '2004', '2005', '2006', '2007', 
         '2008', '2009', '2010','2011', '2012', '2013', 
         '2014', '2015', '2016']] = schmelt.iloc[:,2:].apply(pd.to_numeric, errors='coerce')

# Drop completely NA values
schmelt.dropna(axis=0,subset=['1996', '1997', '1998', '1999', '2000', '2001',
                              '2002', '2003', '2004', '2005', '2006', '2007', 
                              '2008', '2009', '2010','2011', '2012', '2013', 
                              '2014', '2015', '2016'], how='all', inplace=True)

# Preview Data
schmelt.head()

,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,K001,ATTPCTG01,88.76,92.53,90.76,91.04,91.62,91.57,91.88,92.76,...,93.98,93.56,94.45,93.49,93.00,94.86,94.81,93.87,95.12,95.62
1,K001,ATTPCTG02,91.33,91.37,92.66,92.00,92.26,93.87,92.71,93.02,...,93.16,94.99,94.87,95.46,93.83,94.68,94.86,95.12,95.38,95.55
2,K001,ATTPCTG03,92.43,93.11,92.36,93.55,92.90,92.77,94.14,93.56,...,94.57,94.37,95.75,95.55,95.14,95.57,95.35,95.26,95.74,96.32
3,K001,ATTPCTG04,92.36,93.75,92.44,91.88,93.99,93.17,92.38,94.30,...,94.00,94.81,93.66,95.50,95.65,96.35,95.85,95.16,95.67,96.50
4,K001,ATTPCTG05,93.04,92.92,92.61,93.20,92.07,94.12,93.25,93.45,...,94.02,94.43,95.38,94.12,95.74,95.81,96.40,94.99,95.64,95.79


In [3]:
schmelt.tail()

,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
291396,X966,SCHHIGRD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
291397,X966,SCHLEV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,8.0,NaN,NaN,NaN,NaN,NaN
291398,X966,SCHLEV5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN
291399,X966,SCHLOGRD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
291401,X966,SPECHS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN


# 1996 to 2016 data set

In [37]:
# Create Copy of Original Data
schmelt96_16 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt96_16['MinYear'] = np.nan
schmelt96_16['MaxYear'] = np.nan
schmelt96_16['trend96_16'] = np.nan

# Determine Max Years
for cols in schmelt96_16.loc[:,'1996':'2016'].columns:
    schmelt96_16['MaxYear'][schmelt96_16.index[schmelt96_16[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt96_16.loc[:,'2016':'1996':-1].columns:
    schmelt96_16['MinYear'][schmelt96_16.index[schmelt96_16[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt96_16['mean96_16'] = schmelt96_16.loc[:,'1996':'2016'].T.mean()
schmelt96_16['std96_16'] = schmelt96_16.loc[:,'1996':'2016'].T.std()

# Calculate Trend
for cols in schmelt96_16.loc[:,'1996':'2016'].columns:
    for cols2 in schmelt96_16.loc[:,'2016':'1996':-1].columns:
        schmelt96_16['trend96_16'][schmelt96_16['trend96_16'].isnull()] = (schmelt96_16[cols2][schmelt96_16['trend96_16'].isnull()] - 
                                                                           schmelt96_16[cols][schmelt96_16['trend96_16'].isnull()]) / (float(cols2) - float(cols)) 

# Preview 
schmelt96_16.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2012,2013,2014,2015,2016,MinYear,MaxYear,trend96_16,mean96_16,std96_16
0,K001,ATTPCTG01,88.76,92.53,90.76,91.04,91.62,91.57,91.88,92.76,...,94.86,94.81,93.87,95.12,95.62,1996,2016,0.3430,92.978095,1.659119
1,K001,ATTPCTG02,91.33,91.37,92.66,92.00,92.26,93.87,92.71,93.02,...,94.68,94.86,95.12,95.38,95.55,1996,2016,0.2110,93.692381,1.347111
2,K001,ATTPCTG03,92.43,93.11,92.36,93.55,92.90,92.77,94.14,93.56,...,95.57,95.35,95.26,95.74,96.32,1996,2016,0.1945,94.301905,1.204125
3,K001,ATTPCTG04,92.36,93.75,92.44,91.88,93.99,93.17,92.38,94.30,...,96.35,95.85,95.16,95.67,96.50,1996,2016,0.2070,94.230000,1.370482
4,K001,ATTPCTG05,93.04,92.92,92.61,93.20,92.07,94.12,93.25,93.45,...,95.81,96.40,94.99,95.64,95.79,1996,2016,0.1375,94.271905,1.214560


# 2013 to 2016 data set - 4 Year Data Set

In [38]:
# Create Copy of Original Data
schmelt13_16 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt13_16['MinYear'] = np.nan
schmelt13_16['MaxYear'] = np.nan
schmelt13_16['trend13_16'] = np.nan

# Determine Max Years
for cols in schmelt13_16.loc[:,'2013':'2016'].columns:
    schmelt13_16['MaxYear'][schmelt13_16.index[schmelt13_16[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt13_16.loc[:,'2016':'2013':-1].columns:
    schmelt13_16['MinYear'][schmelt13_16.index[schmelt13_16[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt13_16['mean13_16'] = schmelt13_16.loc[:,'2013':'2016'].T.mean()
schmelt13_16['std13_16'] = schmelt13_16.loc[:,'2013':'2016'].T.std()

# Calculate Trend
for cols in schmelt13_16.loc[:,'2013':'2016'].columns:
    for cols2 in schmelt13_16.loc[:,'2016':'2013':-1].columns:
        schmelt13_16['trend13_16'][schmelt13_16['trend13_16'].isnull()] = (schmelt13_16[cols2][schmelt13_16['trend13_16'].isnull()] - 
                                                                           schmelt13_16[cols][schmelt13_16['trend13_16'].isnull()]) / (float(cols2) - float(cols))

# Preview 
schmelt13_16.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2012,2013,2014,2015,2016,MinYear,MaxYear,trend13_16,mean13_16,std13_16
0,K001,ATTPCTG01,88.76,92.53,90.76,91.04,91.62,91.57,91.88,92.76,...,94.86,94.81,93.87,95.12,95.62,2013,2016,0.270000,94.8550,0.736591
1,K001,ATTPCTG02,91.33,91.37,92.66,92.00,92.26,93.87,92.71,93.02,...,94.68,94.86,95.12,95.38,95.55,2013,2016,0.230000,95.2275,0.302145
2,K001,ATTPCTG03,92.43,93.11,92.36,93.55,92.90,92.77,94.14,93.56,...,95.57,95.35,95.26,95.74,96.32,2013,2016,0.323333,95.6675,0.482312
3,K001,ATTPCTG04,92.36,93.75,92.44,91.88,93.99,93.17,92.38,94.30,...,96.35,95.85,95.16,95.67,96.50,2013,2016,0.216667,95.7950,0.553444
4,K001,ATTPCTG05,93.04,92.92,92.61,93.20,92.07,94.12,93.25,93.45,...,95.81,96.40,94.99,95.64,95.79,2013,2016,-0.203333,95.7050,0.578993


# 2009 to 2016 data set - 8 Year Data Set

In [39]:
# Create Copy of Original Data
schmelt09_16 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt09_16['MinYear'] = np.nan
schmelt09_16['MaxYear'] = np.nan
schmelt09_16['trend09_16'] = np.nan

# Determine Max Years
for cols in schmelt09_16.loc[:,'2009':'2016'].columns:
    schmelt09_16['MaxYear'][schmelt09_16.index[schmelt09_16[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt09_16.loc[:,'2016':'2009':-1].columns:
    schmelt09_16['MinYear'][schmelt09_16.index[schmelt09_16[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt09_16['mean09_16'] = schmelt09_16.loc[:,'2009':'2016'].T.mean()
schmelt09_16['std09_16'] = schmelt09_16.loc[:,'2009':'2016'].T.std()

# Calculate Trend
for cols in schmelt09_16.loc[:,'2009':'2016'].columns:
    for cols2 in schmelt09_16.loc[:,'2016':'2009':-1].columns:
        schmelt09_16['trend09_16'][schmelt09_16['trend09_16'].isnull()] = (schmelt09_16[cols2][schmelt09_16['trend09_16'].isnull()] - 
                                                                           schmelt09_16[cols][schmelt09_16['trend09_16'].isnull()]) / (float(cols2) - float(cols))

# Preview 
schmelt09_16.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2012,2013,2014,2015,2016,MinYear,MaxYear,trend09_16,mean09_16,std09_16
0,K001,ATTPCTG01,88.76,92.53,90.76,91.04,91.62,91.57,91.88,92.76,...,94.86,94.81,93.87,95.12,95.62,2009,2016,0.167143,94.40250,0.883010
1,K001,ATTPCTG02,91.33,91.37,92.66,92.00,92.26,93.87,92.71,93.02,...,94.68,94.86,95.12,95.38,95.55,2009,2016,0.097143,94.96875,0.557147
2,K001,ATTPCTG03,92.43,93.11,92.36,93.55,92.90,92.77,94.14,93.56,...,95.57,95.35,95.26,95.74,96.32,2009,2016,0.081429,95.58500,0.368743
3,K001,ATTPCTG04,92.36,93.75,92.44,91.88,93.99,93.17,92.38,94.30,...,96.35,95.85,95.16,95.67,96.50,2009,2016,0.405714,95.54250,0.876939
4,K001,ATTPCTG05,93.04,92.92,92.61,93.20,92.07,94.12,93.25,93.45,...,95.81,96.40,94.99,95.64,95.79,2009,2016,0.058571,95.48375,0.680776


# 2005 to 2016 data set - 12 Year Data Set

In [40]:
# Create Copy of Original Data
schmelt05_16 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt05_16['MinYear'] = np.nan
schmelt05_16['MaxYear'] = np.nan
schmelt05_16['trend05_16'] = np.nan

# Determine Max Years
for cols in schmelt05_16.loc[:,'2005':'2016'].columns:
    schmelt05_16['MaxYear'][schmelt05_16.index[schmelt05_16[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt05_16.loc[:,'2016':'2005':-1].columns:
    schmelt05_16['MinYear'][schmelt05_16.index[schmelt05_16[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt05_16['mean05_16'] = schmelt05_16.loc[:,'2005':'2016'].T.mean()
schmelt05_16['std05_16'] = schmelt05_16.loc[:,'2005':'2016'].T.std()

# Calculate Trend
for cols in schmelt05_16.loc[:,'2005':'2016'].columns:
    for cols2 in schmelt05_16.loc[:,'2016':'2005':-1].columns:
        schmelt05_16['trend05_16'][schmelt05_16['trend05_16'].isnull()] = (schmelt05_16[cols2][schmelt05_16['trend05_16'].isnull()] - 
                                                                           schmelt05_16[cols][schmelt05_16['trend05_16'].isnull()]) / (float(cols2) - float(cols))

# Preview 
schmelt05_16.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2012,2013,2014,2015,2016,MinYear,MaxYear,trend05_16,mean05_16,std05_16
0,K001,ATTPCTG01,88.76,92.53,90.76,91.04,91.62,91.57,91.88,92.76,...,94.86,94.81,93.87,95.12,95.62,2005,2016,0.220909,94.090833,0.865747
1,K001,ATTPCTG02,91.33,91.37,92.66,92.00,92.26,93.87,92.71,93.02,...,94.68,94.86,95.12,95.38,95.55,2005,2016,0.189091,94.615000,0.803894
2,K001,ATTPCTG03,92.43,93.11,92.36,93.55,92.90,92.77,94.14,93.56,...,95.57,95.35,95.26,95.74,96.32,2005,2016,0.204545,95.165833,0.693888
3,K001,ATTPCTG04,92.36,93.75,92.44,91.88,93.99,93.17,92.38,94.30,...,96.35,95.85,95.16,95.67,96.50,2005,2016,0.250909,95.023333,1.084891
4,K001,ATTPCTG05,93.04,92.92,92.61,93.20,92.07,94.12,93.25,93.45,...,95.81,96.40,94.99,95.64,95.79,2005,2016,0.177273,95.016667,0.887564


# 2011 to 2015 data set - Growth Data Set

In [41]:
# Create Copy of Original Data
schmelt11_15 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt11_15['MinYear'] = np.nan
schmelt11_15['MaxYear'] = np.nan
schmelt11_15['trend11_15'] = np.nan

# Determine Max Years
for cols in schmelt11_15.loc[:,'2011':'2015'].columns:
    schmelt11_15['MaxYear'][schmelt11_15.index[schmelt11_15[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt11_15.loc[:,'2015':'2011':-1].columns:
    schmelt11_15['MinYear'][schmelt11_15.index[schmelt11_15[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt11_15['mean11_15'] = schmelt11_15.loc[:,'2011':'2015'].T.mean()
schmelt11_15['std11_15'] = schmelt11_15.loc[:,'2011':'2015'].T.std()

# Calculate Trend
for cols in schmelt11_15.loc[:,'2011':'2015'].columns:
    for cols2 in schmelt11_15.loc[:,'2015':'2011':-1].columns:
        schmelt11_15['trend11_15'][schmelt11_15['trend11_15'].isnull()] = (schmelt11_15[cols2][schmelt11_15['trend11_15'].isnull()] - 
                                                                           schmelt11_15[cols][schmelt11_15['trend11_15'].isnull()]) / (float(cols2) - float(cols))

# Preview 
schmelt11_15.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2012,2013,2014,2015,2016,MinYear,MaxYear,trend11_15,mean11_15,std11_15
0,K001,ATTPCTG01,88.76,92.53,90.76,91.04,91.62,91.57,91.88,92.76,...,94.86,94.81,93.87,95.12,95.62,2011,2015,0.5300,94.332,0.882593
1,K001,ATTPCTG02,91.33,91.37,92.66,92.00,92.26,93.87,92.71,93.02,...,94.68,94.86,95.12,95.38,95.55,2011,2015,0.3875,94.774,0.590407
2,K001,ATTPCTG03,92.43,93.11,92.36,93.55,92.90,92.77,94.14,93.56,...,95.57,95.35,95.26,95.74,96.32,2011,2015,0.1500,95.412,0.241599
3,K001,ATTPCTG04,92.36,93.75,92.44,91.88,93.99,93.17,92.38,94.30,...,96.35,95.85,95.16,95.67,96.50,2011,2015,0.0050,95.736,0.428229
4,K001,ATTPCTG05,93.04,92.92,92.61,93.20,92.07,94.12,93.25,93.45,...,95.81,96.40,94.99,95.64,95.79,2011,2015,-0.0250,95.716,0.502524
